In [ ]:
# default_exp model.rnnvae

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export

import torch
from torch import nn, optim
from deeptool.architecture import Encoder, Decoder
from deeptool.utils import Tracker

# RNN VAE

> Structure for an Approach maintained a pseudo space realted realtion 

In [ ]:
# load some test dataset to confirm architecture:
from deeptool.parameters import get_all_args
args = get_all_args()
args

Namespace(alpha=1, batch_size=16, beta=1, bi_extension=False, classes=['abn', 'acl', 'men'], crop_percent=0.7, crop_size=16, dim=3, gamma=5, lam=10, load_model=False, lr=0.0001, m=2000, min_size=4, model_path='./data/src/_model', model_type='diagnosis', mrnet_backbone='alexnet', mrnet_batch_update=32, mrnet_hidden_dim=256, mrnet_label_smoothing=0.1, mrnet_rnn_gap=True, mrnet_singlestream=True, n_epochs=500, n_fea_down=16, n_fea_up=16, n_gpu=1, n_pretrain=10, n_res2d=0, n_res3d=0, n_z=100, num_worker=0, p_drop=0.1, perspectives=['axial', 'coronal', 'sagittal'], pic_size=256, rand_crop=True, root_dir='C:/Users/ga46yeg/data/MRNet-v1.0', scale2d=2, scale3d=2, track=True, vq_beta=0.25, vq_class=1.0, vq_classify=False, vq_gamma=0.99, vq_gan=False, vq_layers=[8], vq_n_embed=512, watch_batch=1130, wgan=True)

In [ ]:
# export

class Encoder2D(nn.Module):
    """
    Simple class containing the 2D encoder architecture
    """
    
    def __init__(self, ):
        """
        
        """

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_RNN_VAE.ipynb.
Converted 22_vqvae.ipynb.
Converted 99_index.ipynb.
